In [42]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from tqdm.auto import trange
import pickle
import re
import gc
import open_clip

In [5]:
!pip install open_clip_torch -U

In [6]:
open_clip.list_pretrained()

[('RN50', 'openai'),
 ('RN50', 'yfcc15m'),
 ('RN50', 'cc12m'),
 ('RN101', 'openai'),
 ('RN101', 'yfcc15m'),
 ('RN50x4', 'openai'),
 ('RN50x16', 'openai'),
 ('RN50x64', 'openai'),
 ('ViT-B-32', 'openai'),
 ('ViT-B-32', 'laion400m_e31'),
 ('ViT-B-32', 'laion400m_e32'),
 ('ViT-B-32', 'laion2b_e16'),
 ('ViT-B-32', 'laion2b_s34b_b79k'),
 ('ViT-B-32', 'datacomp_xl_s13b_b90k'),
 ('ViT-B-32', 'datacomp_m_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_clip_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_laion_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_image_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_text_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_basic_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_s128m_b4k'),
 ('ViT-B-32', 'datacomp_s_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_clip_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_laion_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_image_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_text_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_basic_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_s13m_b4k'),
 ('ViT-

In [12]:
model, transform = open_clip.create_model_from_pretrained("ViT-L-14-quickgelu", "dfn2b", device="cuda")
tokenizer = open_clip.get_tokenizer("ViT-L-14-quickgelu")

open_clip_pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


NameError: name 'get_tokenizer' is not defined

In [13]:
tokenizer = open_clip.get_tokenizer("ViT-L-14-quickgelu")

In [24]:
model

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
    (patch_dropout): Identity()
    (ln_pre): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0-23): 24 x ResidualAttentionBlock(
          (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
          )
          (ls_1): Identity()
          (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
          )
          (ls_2): Identity()
        )
      )
    )
    (ln_post): LayerNorm((1024,), eps=1e-05, elementwise_affine=Tru

In [25]:
!wget https://huggingface.co/datasets/andrey200702/REC_SYS/resolve/main/graph.pkl?download=true

--2024-11-01 00:28:18--  https://huggingface.co/datasets/andrey200702/REC_SYS/resolve/main/graph.pkl?download=true
Resolving huggingface.co (huggingface.co)... 3.168.73.106, 3.168.73.129, 3.168.73.111, ...
Connecting to huggingface.co (huggingface.co)|3.168.73.106|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/a0/67/a06735d846971cc734adc3f95dcaef40fbf6a44a0ece4321d3e191c0c4e01d03/68018e4aa848131fac8d4000c4fed783cac6fabb7d65498f94789a0f1944e570?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27graph.pkl%3B+filename%3D%22graph.pkl%22%3B&Expires=1730680098&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMDY4MDA5OH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2EwLzY3L2EwNjczNWQ4NDY5NzFjYzczNGFkYzNmOTVkY2FlZjQwZmJmNmE0NGEwZWNlNDMyMWQzZTE5MWMwYzRlMDFkMDMvNjgwMThlNGFhODQ4MTMxZmFjOGQ0MDAwYzRmZWQ3ODNjYWM2ZmFiYjdkNjU0OThmOTQ3ODlhMGYxOTQ0ZTU3MD9yZXNwb25zZS1j

In [26]:
graph_state = pickle.load(open('graph.pkl','rb'))

In [39]:
concepts = list(graph_state['concepts_maper'].keys())
concepts = [x if type(x) == str else '' for x in concepts]

In [51]:
def prepare_text(s):
    return re.sub(" +", " ",s.lower().replace('_',' ').strip())

@torch.no_grad()
def create_embeds(texts,model,tokenizer,batch_size=64,max_length=77):
    features = []
    texts = [prepare_text(x) for x in texts]
    for i in trange(len(texts) // batch_size + 1):
        encs = tokenizer(texts[i*batch_size:(i+1)*batch_size],context_length=max_length)
        embeds = model.encode_text(encs.cuda())
        embeds = torch.nn.functional.normalize(embeds, p=2, dim=1).detach().cpu().numpy()
        features.append(embeds)
    return np.concatenate(features,dtype=np.float32)

In [ ]:
embeds = create_embeds(concepts,model,tokenizer,batch_size=1024)

  0%|          | 0/781 [00:00<?, ?it/s]